In [1]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
yf.pdr_override()
df = pdr.get_data_yahoo(["JPM"])

[*********************100%%**********************]  1 of 1 completed


In [3]:
close=df.Close['2023-5-1':'2024-5-1']
log_ret=np.log(close)-np.log(close.shift(1))
mean = log_ret.mean()*252
std = log_ret.std()*np.sqrt(252)
rate=5.5/100
time=1
n =100
u=np.exp(std*np.sqrt(time/n))
d=np.exp(-std*np.sqrt(time/n))
p=(np.exp(rate*time/n)-d)/(u-d)

print("u=",u)
print("d=",d)
print("p=",p)

u= 1.0170334270833978
d= 0.9832518512864956
p= 0.5120631463504347


In [4]:
close.tail()

Date
2024-04-25    193.369995
2024-04-26    193.490005
2024-04-29    193.279999
2024-04-30    191.740005
2024-05-01    191.860001
Name: Close, dtype: float64

In [5]:
s0= 191.860001

In [6]:
import numpy as np
def binomial_tree(n,d,u,S0):
    stock_tree = np.zeros((2*n-1,n))
    lst_ = list(range(n,2*n-1))
    
    for i in range(n+1):
            for j in range(i+1):
                if i+j ==n-1:
                    stock_tree[i][j]=S0*(u)**j
                    stock_tree[j][i]=S0*(u)**i
    for i in range(len(lst_)):
     
        stock_tree[lst_[i]][i+1]=S0*(d)**(i+1)
        
    for i in range(2*n-1):
        for j in range(n):
            if stock_tree[i][j]!=0 and j+2<n:
                stock_tree[i][j+2]=stock_tree[i][j]
        
    return np.round(stock_tree,3)

In [7]:
binomial_tree(n,d,u,s0)

array([[   0.   ,    0.   ,    0.   , ...,    0.   ,    0.   , 1021.344],
       [   0.   ,    0.   ,    0.   , ...,    0.   , 1004.238,    0.   ],
       [   0.   ,    0.   ,    0.   , ...,  987.419,    0.   ,  987.419],
       ...,
       [   0.   ,    0.   ,    0.   , ...,   37.279,    0.   ,   37.279],
       [   0.   ,    0.   ,    0.   , ...,    0.   ,   36.655,    0.   ],
       [   0.   ,    0.   ,    0.   , ...,    0.   ,    0.   ,   36.041]])

In [8]:
def call_europe(n,d,u,s0,k,rate,time):
    pricing = binomial_tree(n,d,u,s0)
    matrix1 = np.zeros((pricing.shape))
    #k=120 #strike price
    price_pred = pricing
    for i in range(pricing.shape[0]):
    
        matrix1[i][0]=max(price_pred[i][-1]-k,0)
    matrix2=matrix1

    p=(np.exp(rate*time/(n-1))-d)/(u-d)
    for i in range(1,matrix2.shape[1]):
        for j in range(1,matrix2.shape[0]-1):
            if j%2==0 and i%2==0 and j>=i:
                if j+i<=(pricing.shape[1]-1)*2:# and j+i>=min(i,j)*2:
                    #print(j,i,j+1,j-1,i-1)
                    matrix2[j,i]=((1-p)*matrix2[j+1,i-1]+(p)*matrix2[j-1,i-1])*np.exp(-rate*time/(n-1))
            elif j%2!=0 and i%2!=0 and j>=i:
                if j+i<=(pricing.shape[1]-1)*2:# and j+i>=min(i,j)*2:
                    #print(j,i,j+1,j-1,i-1)
                    matrix2[j,i]=(matrix2[j+1,i-1]*(1-p)+(p)*matrix2[j-1,i-1])*np.exp(-rate*time/(n-1))

    print('price call european is', matrix2[n-1,-1])

In [9]:
call_europe(n+1,d,u,s0,190,rate,time)

price call european is 19.44211043229868


In [10]:
def eu_call_barrier(n,d,u,s0,k,rate,time):
    barrier = 0.4567*s0
    print('barrier is', barrier)
    pricing = binomial_tree(n,d,u,s0)
    for i in range(pricing.shape[0]):
        for j in range(pricing.shape[1]):
            if pricing[i][j]<barrier:
                pricing[i][j]=0
    matrix1 = np.zeros((pricing.shape))
    #k=120 #strike price
    price_pred = pricing
    for i in range(pricing.shape[0]):
    
        matrix1[i][0]=max(price_pred[i][-1]-k,0)
    matrix2=matrix1

    p=(np.exp(rate*time/(n-1))-d)/(u-d)
    for i in range(1,matrix2.shape[1]):
        for j in range(1,matrix2.shape[0]-1):
            if j%2==0 and i%2==0 and j>=i:
                if j+i<=(pricing.shape[1]-1)*2:# and j+i>=min(i,j)*2:
                    #print(j,i,j+1,j-1,i-1)
                    matrix2[j,i]=((1-p)*matrix2[j+1,i-1]+(p)*matrix2[j-1,i-1])*np.exp(-rate*time/(n-1))
            elif j%2!=0 and i%2!=0 and j>=i:
                if j+i<=(pricing.shape[1]-1)*2:# and j+i>=min(i,j)*2:
                    #print(j,i,j+1,j-1,i-1)
                    matrix2[j,i]=(matrix2[j+1,i-1]*(1-p)+(p)*matrix2[j-1,i-1])*np.exp(-rate*time/(n-1))
    print('price call european w/ barrier is', matrix2[n-1,-1])

In [11]:
eu_call_barrier(n+1,d,u,s0,190,rate,time)

barrier is 87.6224624567
price call european w/ barrier is 19.44211043229868


In [12]:
def eu_put(n,d,u,s0,k,rate,time):
    pricing = binomial_tree(n,d,u,s0)
    eu_put = np.zeros((pricing.shape[0],pricing.shape[1]))
    for i in range(len(eu_put)):
        if pricing[i][-1]!=0:
            eu_put[i][0]=max(k-pricing[i][-1],0)
        else:
            eu_put[i][0]=0
    matrix2=eu_put

    p=(np.exp(rate*time/(n-1))-d)/(u-d)
    for i in range(1,matrix2.shape[1]):
        for j in range(1,matrix2.shape[0]-1):
            if j%2==0 and i%2==0 and j>=i:
                if j+i<=(pricing.shape[1]-1)*2:# and j+i>=min(i,j)*2:
                    #print(j,i,j+1,j-1,i-1)
                    matrix2[j,i]=((1-p)*matrix2[j+1,i-1]+(p)*matrix2[j-1,i-1])*np.exp(-rate*time/(n-1))
            elif j%2!=0 and i%2!=0 and j>=i:
                if j+i<=(pricing.shape[1]-1)*2:# and j+i>=min(i,j)*2:
                    #print(j,i,j+1,j-1,i-1)
                    matrix2[j,i]=(matrix2[j+1,i-1]*(1-p)+(p)*matrix2[j-1,i-1])*np.exp(-rate*time/(n-1))
    #print(p)
    print("price of eu put is", matrix2[n-1,-1])

In [13]:
eu_put(n+1,d,u,s0,190,rate,time)

price of eu put is 7.4142819935709054


In [17]:
def eu_put_barrier(n,d,u,s0,k,rate,time):
    pricing = binomial_tree(n,d,u,s0)
    barrier = 0.4567*s0 #170
    print('barrier is', barrier)
    for i in range(pricing.shape[0]):
        for j in range(pricing.shape[1]):
            if pricing[i][j]<barrier:
                pricing[i][j]=0
    eu_put = np.zeros((pricing.shape[0],pricing.shape[1]))
    for i in range(len(eu_put)):
        if pricing[i][-1]!=0:
            eu_put[i][0]=max(k-pricing[i][-1],0)
        else:
            eu_put[i][0]=0
    matrix2=eu_put

    p=(np.exp(rate*time/(n-1))-d)/(u-d)
    for i in range(1,matrix2.shape[1]):
        for j in range(1,matrix2.shape[0]-1):
            if j%2==0 and i%2==0 and j>=i:
                if j+i<=(pricing.shape[1]-1)*2:# and j+i>=min(i,j)*2:
                    #print(j,i,j+1,j-1,i-1)
                    matrix2[j,i]=((1-p)*matrix2[j+1,i-1]+(p)*matrix2[j-1,i-1])*np.exp(-rate*time/(n-1))
            elif j%2!=0 and i%2!=0 and j>=i:
                if j+i<=(pricing.shape[1]-1)*2:# and j+i>=min(i,j)*2:
                    #print(j,i,j+1,j-1,i-1)
                    matrix2[j,i]=(matrix2[j+1,i-1]*(1-p)+(p)*matrix2[j-1,i-1])*np.exp(-rate*time/(n-1))
    #print(p)
    print("price of eu put w/ barrier is", matrix2[n-1,-1])

In [18]:
eu_put_barrier(n+1,d,u,s0,190,rate,time)

barrier is 87.6224624567
price of eu put w/ barrier is 7.414257062237618


In [19]:
def am_call(n,d,u,s0,k,rate,time):
    price = binomial_tree(n,d,u,s0)[:,-1]
    pricing = binomial_tree(n,d,u,s0)
    payoff=[]
    for i in range(len(price)):
        if price[i]==0:
            payoff.append(0)
        else:
            payoff.append(max(price[i]-k,0))
    p=(np.exp(rate*time/(n-1))-d)/(u-d)
    am_call = np.zeros((2*n-1,n))
    for i in range(len(payoff)):
        am_call[i][0]=payoff[i]
        
    for i in range(1,am_call.shape[1]):
        for j in range(1,am_call.shape[0]-1):
            if j+i<=(n-1)*2 and pricing[j][pricing.shape[1]-i-1]!=0:
                am_call[j,i]=max(((1-p)*am_call[j+1,i-1]+(p)*am_call[j-1,i-1])*np.exp(-rate*time/(n-1)),
                                 max(pricing[j][pricing.shape[1]-i-1]-k,0))

    print('price of american call is',am_call[n-1,-1])

In [20]:
am_call(n+1,d,u,s0,190,rate,time)

price of american call is 19.44211043229868


In [21]:
def am_call_barrier(n,d,u,s0,k,rate,time):
    price = binomial_tree(n,d,u,s0)[:,-1]
    pricing = binomial_tree(n,d,u,s0)
    barrier = 0.4567*s0  
    #print(np.round(pricing,1))
    print('barrier is',barrier)
    for i in range(pricing.shape[0]):
        for j in range(pricing.shape[1]):
            if pricing[i][j]<barrier:
                pricing[i][j]=0
    #print(np.round(pricing,1))
    payoff=[]
    for i in range(len(price)):
        if price[i]==0 and price[i]<barrier:
            payoff.append(0)
        else:
            payoff.append(max(price[i]-k,0))
    #print(payoff)
    p=(np.exp(rate*time/(n-1))-d)/(u-d)
    am_call = np.zeros((2*n-1,n))
    for i in range(len(payoff)):
        am_call[i][0]=payoff[i]
        
    for i in range(1,am_call.shape[1]):
        for j in range(1,am_call.shape[0]-1):
            if j+i<=(n-1)*2 and pricing[j][pricing.shape[1]-i-1]!=0:
                am_call[j,i]=max(((1-p)*am_call[j+1,i-1]+(p)*am_call[j-1,i-1])*np.exp(-rate*time/(n-1)),
                                 max(pricing[j][pricing.shape[1]-i-1]-k,0))
    print('price of american call w/ barrier is',am_call[n-1,-1])

In [22]:
am_call_barrier(n+1,d,u,s0,190,rate,time)

barrier is 87.6224624567
price of american call w/ barrier is 19.44211043229868


In [23]:
def am_put(n,d,u,s0,k,rate,time):
    price = binomial_tree(n,d,u,s0)[:,-1]
    pricing = binomial_tree(n,d,u,s0)
    payoff=[]
    for i in range(len(price)):
        if price[i]==0:
            payoff.append(0)
        else:
            payoff.append(max(k-price[i],0))
    p=(np.exp(rate*time/(n-1))-d)/(u-d)
    am_put = np.zeros((2*n-1,n))
    for i in range(len(payoff)):
        am_put[i][0]=payoff[i]
        
    for i in range(1,am_put.shape[1]):
        for j in range(1,am_put.shape[0]-1):
            if j+i<=(n-1)*2 and pricing[j][pricing.shape[1]-i-1]!=0:
                am_put[j,i]=max(((1-p)*am_put[j+1,i-1]+(p)*am_put[j-1,i-1])*np.exp(-rate*time/(n-1)),
                                max(k-pricing[j][pricing.shape[1]-i-1],0))

    print('price of american put is',am_put[n-1,-1])

In [24]:
am_put(n+1,d,u,s0,190,rate,time)

price of american put is 8.383400092861265


In [25]:
def am_put_barrier(n,d,u,s0,k,rate,time):
    price = binomial_tree(n,d,u,s0)[:,-1]
    pricing = binomial_tree(n,d,u,s0)
    barrier = 0.4567*s0
    print('barrier is',barrier)
    for i in range(pricing.shape[0]):
        for j in range(pricing.shape[1]):
            if pricing[i][j]<barrier:
                pricing[i][j]=0
    payoff=[]
    for i in range(len(price)):
        if price[i]==0 or price[i]<barrier:
            payoff.append(0)
        else:
            payoff.append(max(k-price[i],0))
            
    p=(np.exp(rate*time/(n-1))-d)/(u-d)
    am_put = np.zeros((2*n-1,n))
    for i in range(len(payoff)):
        am_put[i][0]=payoff[i]
        
    for i in range(1,am_put.shape[1]):
        for j in range(1,am_put.shape[0]-1):
            if j+i<=(n-1)*2 and pricing[j][pricing.shape[1]-i-1]!=0:
                am_put[j,i]=max(((1-p)*am_put[j+1,i-1]+(p)*am_put[j-1,i-1])*np.exp(-rate*time/(n-1)),
                                max(k-pricing[j][pricing.shape[1]-i-1],0))
    print('price of american put w/ barrier is',am_put[n-1,-1])

In [26]:
am_put_barrier(n+1,d,u,s0,190,rate,time)

barrier is 87.6224624567
price of american put w/ barrier is 8.383400092861265
